In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, TimeDistributed, Bidirectional

ribasti = pd.read_csv('RNN.csv')

buckArab = {"'":"ء", "|":"آ", "?":"أ", "&":"ؤ", "<":"إ", "}":"ئ", "A":"ا", "b":"ب", "p":"ة", "t":"ت", "v":"ث", "g":"ج", "H":"ح", "x":"خ", "d":"د", "*":"ذ", "r":"ر", "z":"ز", "s":"س", "$":"ش", "S":"ص", "D":"ض", "T":"ط", "Z":"ظ", "E":"ع", "G":"غ", "_":"ـ", "f":"ف", "q":"ق", "k":"ك", "l":"ل", "m":"م", "n":"ن", "h":"ه", "w":"و", "Y":"ى", "y":"ي", "F":"ً", "N":"ٌ", "K":"ٍ", "~":"ّ", "o":"ْ", "u":"ُ", "a":"َ", "i":"ِ", "{":"ا"}

def transString(string, reverse=0):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''

    for k, v in buckArab.items():
        if not reverse:
            string = string.replace(v, k)
        else:
            string = string.replace(k, v)

    return string

korpus = []
for _ in ribasti['0']:
  korpus.append(str(_))
korpusan = " ".join(korpus)

korpus_harakat = []
for _ in ribasti['1']:
  korpus_harakat.append(str(_))
korpusan_harakat = " ".join(korpus_harakat)

korpus2 = []
korpusan2 = korpusan.split()
for i in range(len(korpusan2)-7):
  korpus2.append(" ".join(korpusan2[i:i+7]))

korpus2_h = []
korpusan2_h = korpusan_harakat.split()
for i in range(len(korpusan2_h)-7):
  korpus2_h.append(" ".join(korpusan2_h[i:i+7]))

token_x = Tokenizer()
token_y = Tokenizer()
token_x.fit_on_texts(korpus2)
token_y.fit_on_texts(korpus2_h)
x_seq = token_x.texts_to_sequences(korpus2)
y_seq = token_y.texts_to_sequences(korpus2_h)
x_seq_padding = pad_sequences(x_seq, maxlen=7, padding="post")
x = np.asarray(x_seq_padding).astype(float)
y_seq_padding = pad_sequences(y_seq, maxlen=7, padding="post")
y =  np.asarray(y_seq_padding)

embedding_dim = 16
classifier = Sequential()
classifier.add(Embedding(int(np.max(x)+1), embedding_dim, input_length=x.shape[1], input_shape=x.shape[1:]))
classifier.add(Bidirectional(LSTM(128, return_sequences=True)))
classifier.add(TimeDistributed(Dense(64, activation='relu')))
classifier.add(TimeDistributed(Dense(int(np.max(y)+1), activation='softmax')))
classifier.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.load_weights("ribasti.h5")
print("Masukan teks bahasa arab gundul : ")
kalimat = input()
kalimat_ = token_x.texts_to_sequences([kalimat])
kalimat_pad = pad_sequences(kalimat_, maxlen=7, padding="post")
pred = classifier.predict(kalimat_pad)
pred_ = [(np.argmax(pred[0][0])), (np.argmax(pred[0][1])), (np.argmax(pred[0][2])), (np.argmax(pred[0][3])), (np.argmax(pred[0][4])), (np.argmax(pred[0][5])), (np.argmax(pred[0][6]))]
pred__ = []
for i in pred_:
  pred__.append([i])
hasil_ = " ".join(token_y.sequences_to_texts(pred__))
kalimat__ = transString (kalimat, reverse=1)
hasil__ = transString (hasil_, reverse=1)
print(kalimat__)
print(hasil_)
print(hasil__)

Masukan teks bahasa arab gundul : 
b sm {ll~h {l r~Hm`n {l r~Hym
ب سم اللّه ال رّحم`ن ال رّحيم
l r ahoma ni l r ahiymi
ل ر َهْمَ نِ ل ر َهِيمِ
